<a href="https://colab.research.google.com/github/basugautam/Reproducibility-Challenge-Project/blob/Architecture-Files/10_rethinkig_error_aggregation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import Required Libraries
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import Dataset, DataLoader

# 🔹 Step 1: Load and Preprocess Dataset
dataset_path = r"C:\LCTSF\Dataset\timeseries_dataset.csv"
df = pd.read_csv(dataset_path)

# Extract time-series values
data = df['value'].values.reshape(-1, 1)

# Normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data)

# 🔹 Step 2: Create Time-Series Sequences
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length, 0])
        y.append(data[i + seq_length, 0])
    return np.array(X), np.array(y)

sequence_length = 50
X, y = create_sequences(data_scaled, sequence_length)

# Split into Training & Testing Sets
split_idx = int(0.8 * len(X))
X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]

# Convert data to PyTorch tensors
X_train_torch = torch.tensor(X_train, dtype=torch.float32).unsqueeze(-1)  # Add feature dimension
y_train_torch = torch.tensor(y_train, dtype=torch.float32).unsqueeze(-1)

X_test_torch = torch.tensor(X_test, dtype=torch.float32).unsqueeze(-1)
y_test_torch = torch.tensor(y_test, dtype=torch.float32).unsqueeze(-1)

# Create PyTorch Dataset
class TimeSeriesDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Define DataLoaders
batch_size = 32
train_loader = DataLoader(TimeSeriesDataset(X_train_torch, y_train_torch), batch_size=batch_size, shuffle=True)
test_loader = DataLoader(TimeSeriesDataset(X_test_torch, y_test_torch), batch_size=batch_size, shuffle=False)

# 🔹 Step 3: Define Transformer Model with Weighted Error Aggregation
class TransformerForecasting(nn.Module):
    def __init__(self, input_dim, model_dim, num_heads, num_layers, dropout=0.1):
        super(TransformerForecasting, self).__init__()
        self.embedding = nn.Linear(input_dim, model_dim)  # Project input to model dimension
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=model_dim, nhead=num_heads, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(model_dim, 1)  # Final prediction layer
        self.weight_vector = nn.Parameter(torch.linspace(1.0, 2.0, sequence_length))  # Learnable weights for time-step importance

    def forward(self, x):
        x = self.embedding(x)
        x = self.transformer_encoder(x)
        output = self.fc(x[:, -1, :])  # Use last time step for prediction
        return output

# Instantiate Transformer Model
input_dim = 1
model_dim = 64
num_heads = 4
num_layers =_
